# Python DS-ao-Dev

## Bussiness Problem

**Star Jeans Company**

- Eduardo e Marcelo são dois brasileiros, amigos e sócios de empreendimento. Depois de vários negócio bem sucedidos, eles estão planejando entrar no mercado de moda dos USA como um modelo de negócio do tipo E-commerce.

- A idéia inicial é entrar no mercado com apenas um produto e para um público específico, no caso o produto seria calças Jenas para o público masculino. O objetivo é manter o custo de operação baixo e escalar a medida que forem conseguindo clientes.

- Porém, mesmo com o produto de entrada e a audiência definidos, os dois sócios não tem experiência nesse mercado de moda e portanto não sabem definir coisas básicas como preço, o tipo de calça e o material para a fabricação de cada peça.

- Assim, os dois sócios contrataram uma consultoria de Ciência de Dados para responder as seguintes perguntas: ]
    1. Qual o melhor preço de venda para as calças? 
    2. Quantos tipos de calças e suas cores para o produto inicial? 
    3. Quais as matérias-prima necessárias para confeccionar as calças?
    
- As principais concorrentes da empresa Start Jeans são as americadas H&M e Macys.

## Solution Planning (Input-Output-Tasks)

**Bussiness Question**

- Which is the best price for jeans?

1. Input:
    1. Fonte de dados
        - Site da H&M: https://www2.hm.com/en_us/men/products/jeans.html
        - Site da Macys: https://www.macys.com/shop/mens-clothing/mens-jeans
    2. Ferramentas
        - Python 3.8.0
        - Bibliotecas de Webscrapping ( BS4, Selenium )
        - PyCharm
        - Jupyter Notebook ( Analise e prototipagens )
        - Crontjob, Airflow
        - Streamlit
    
2. Output:
    1. A resposta para a pergunta.
        - Mediana dos preços dos concorrents.
    2. Formato da entrega
        - Tabela ou gráfico
    3. Local da entrega
        - App Streamlit
    
3. Tasks:
    1. Passo a passso para construir o cálculo da mediana ou média
        - Realizar o calculo da mediana sobre o produto, tipo e cor
    2. Definir o formato da entrega ( Visualização, Tabela, Frase )
        - Gráfico de barras com a mediana dos preço dos produtos, por tipo e cor dos últimos 30 dia
        - Tabela com as seguintes colunas: id | product_name | product_type | product_color | produ
        - Definição do schema: Colunas e seu tipo
        - Definição a infraestrutura de armazenamento ( SQLITE3 )
        - Design do ETL ( Scripts de Extração, Transformação e Carga )
        - Planejamento de Agendamento dos scripts ( dependencias entre os scripts )
        - Fazer as visualizações
        - Entrega do produto final
    3. Decidir o local de entrega ( PowerBi, Telegram, Email, Streamlit, Intranet ),
        - App com Streamlit

## Bussiness Models

“Como você planeja ganhar dinheiro”, Michael Lewis

“Um modelo de negócio descreve a lógica de criação, entrega e captura de valor por
parte de uma organização”, Alexander Osterwalder

- E-commerce:
    1. Faturamento: Vendas de um produto.
    2. Exemplo: Lojas Riachuelo, Submarino, Magazine Luiza, etc
        
- Software AS a Service ( SaaS ):
    1. Faturamento: Assinatura mensal/anual de utilização ou por usuário.
    2. Exemplo: Looker, Asana, Gmail, Salesforce.
    
- Serviço:
    1. Faturamento: Prestação de serviço por tempo ou projeto.
    2. Exemplo: Sul América, Porto Seguro, Mapfre.
    
- Mobile App:
    1. Faturamento: Venda de upgrades.
    2. Exemplo: Wildlife, Ubisoft, Games Mobile.
    
- Media Site:
    1. Faturamento: Cobrança por clicks ou visualizações de um determinado anúncio.
    2. Exemplo: Facebook, Google, UOL, G1, etc.
    
- Marketplace:
    1. Faturamento: Taxa sobre a transação entre o passageiro e o motorista.
    2. Exemplo: Uber, Ifood, 99, Elo7, Submarino.

## E-commerce Metrics

- **Growth Metrics**:
    1. Porcentagem do Marketshare
    2. Número de Clientes Novos
- **Revenue Metrics**:
    1. Número de Vendas
    2. Ticket Médio
    3. LTV ( Long Time Value )
    4. Recência Média
    5. Basket Size Médio
    6. Markup médio
- **Cost Metrics**:
    1. CAC ( Custo de aquisição de Clientes )
    2. Desconto médio
    3. Custo de Produção
    4. Taxa de devolução
    5. Custos Fixos ( Folha de pagamento, escritório, softwares )
    6. Impostos

In [ ]:
from IPython.display import Image
Image(filename='/home/marxcerqueira/repos/Data-Science-Projects/pa005_insiders_clustering/pa005_marx_cerqueira/reports/figures/mapa_metricas_e_commerce.png')

# Imports

In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
pwd

'/home/marxcerqueira/repos/python-ds-ao-dev'

# Loading Data (Web Scrapping)

In [349]:
#load data
data = pd.read_csv( '/home/marxcerqueira/repos/python-ds-ao-dev/products_hm.csv' )
#drop unnecessary column
data = data.drop('Unnamed: 0', axis = 1)

#rename columns
data = data.rename(columns = {'Fit': 'fit', 'Composition': 'composition','Size': 'size' ,'Product safety': 'product_safety'} )

#####______#####
#product name
data['product_name'] = data['product_name'].apply(lambda x: x.replace(' ', '_').lower())

#product price
data['product_price'] = data['product_price'].apply(lambda x: x.replace('$', '')).astype(float)

#scrapy datetime
data['scrapy_datetime'] = pd.to_datetime(data['scrapy_datetime'], format = '%Y-%m-%d %H:%M:%S')

#style_id
#color_id
#color name
data['color_name'] = data['color_name'].apply(lambda x: x.replace(' ', '_').replace('/', '_').lower() if pd.notnull(x) else x)

#fit
data['fit'] = data['fit'].apply(lambda x: x.replace(' ', '_').lower() if pd.notnull(x) else x)

#composition


#====  size  ======
#size number
data['size_number'] = data['size'].apply(lambda x: re.search('\d{3}cm', x).group(0) if pd.notnull(x) else x)
data['size_number'] = data['size_number'].apply(lambda x: re.search('\d+', x).group(0) if pd.notnull(x) else x) #group(0) locates the whole match expression

#size model 
data['size_model'] = data['size'].str.extract('(\d+/\\d+)') #.str to vectorize the lines, .extracts cant be applied in the whole column

#product safety


#composition
data = data[~data['composition'].str.contains('Pocket lining:', na = False)] #na = false pula os NAs existentes
data = data[~data['composition'].str.contains('Shell:', na = False)]
data = data[~data['composition'].str.contains('Lining:', na = False)] 

#drop duplicates
data = data.drop_duplicates(subset = ['product_id', 'product_category', 'product_name', 'product_price',
       'scrapy_datetime', 'style_id', 'color_id', 'color_name', 'fit'], keep = 'last')

#reset index
data = data.reset_index( drop = True)

#break composition by comma
df1 = data['composition'].str.split(',', expand = True)

# cotton / polyester / spandex / estasterel
# creating empty dataframe as reference to organize the wanted columns and 
# then concatanete with the main dataframe, but it has to have the same lenght as 'data' dataframe

df_ref = pd.DataFrame(index = np.arange(len(data)), columns = ['cotton', 'polyester', 'spandex', 'elasterell'])

# #cotton
df_cotton = df1[0]
df_cotton.name = 'cotton'

# # df_ref = pd.merge(df_ref, df_cotton, left_index=True, right_index=True)

df_ref = pd.concat([df_ref, df_cotton], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep = 'last')] #exclue colunas duplicadas, mantem as diferentes
df_ref['cotton'] = df_ref['cotton'].fillna('Cotton 0%')

# # #polyester
df_polyester = df1.loc[df1[1].str.contains('Polyester', na = True), 1]
df_polyester.name = 'polyester'

df_ref = pd.concat([df_ref, df_polyester], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['polyester'] = df_ref['polyester'].fillna('polyester 0%')


# # #spandex
df_spandex = df1.loc[df1[1].str.contains('Spandex', na = True), 1]
df_spandex.name = 'spandex'

# df_spandex2 = df1.loc[df1[2].str.contains('Spandex', na = True), 2]
# df_spandex2.name = 'spandex'

#other solution
df_spandex = df_spandex.combine_first(df1[2])

df_ref = pd.concat([df_ref, df_spandex], axis = 1)
# df_ref = pd.concat([df_ref, df_spandex2], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['spandex'] = df_ref['spandex'].fillna('spandex 0%')

# # #elasterell
df_elasterell = df1.loc[df1[1].str.contains('Elasterell', na = True), 1]
df_elasterell.name = 'elasterell'

df_ref = pd.concat([df_ref, df_elasterell], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['elasterell'] = df_ref['elasterell'].fillna('elasterell 0%')

# # #final join
data = pd.concat([data, df_ref], axis = 1)

# #formatt composition data
data['cotton'] = data['cotton'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
data['polyester'] = data['polyester'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
data['spandex'] = data['spandex'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
data['elasterell'] = data['elasterell'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)


#drop columns
data = data.drop(columns = ['size', 'product_safety','composition'], axis = 1)

#drop duplicates

data = data.drop_duplicates()

data.head(25)

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,color_name,fit,size_number,size_model,cotton,polyester,spandex,elasterell
0,690449043,men_jeans_ripped,skinny_jeans,39.99,2021-12-13 13:18:24,690449,43,black_trashed,skinny_fit,NaN,NaN,0.98,0.0,0.02,0.0
1,690449043,men_jeans_ripped,skinny_jeans,39.99,2021-12-13 13:18:24,690449,43,denim_blue,skinny_fit,184,31/32,0.98,0.0,0.02,0.0
2,690449043,men_jeans_ripped,skinny_jeans,39.99,2021-12-13 13:18:24,690449,43,black_washed_out,skinny_fit,184,31/32,0.98,0.0,0.02,0.0
3,690449043,men_jeans_ripped,skinny_jeans,39.99,2021-12-13 13:18:24,690449,43,white,skinny_fit,184,31/32,0.98,0.0,0.02,0.0
4,690449043,men_jeans_ripped,skinny_jeans,39.99,2021-12-13 13:18:24,690449,43,black_washed,skinny_fit,184,31/32,0.98,0.0,0.02,0.0
5,690449043,men_jeans_ripped,skinny_jeans,39.99,2021-12-13 13:18:24,690449,43,dark_denim_blue_trashed,skinny_fit,184,31/32,0.98,0.0,0.02,0.0
6,690449043,men_jeans_ripped,skinny_jeans,39.99,2021-12-13 13:18:24,690449,43,dark_blue_trashed,skinny_fit,184,31/32,0.98,0.0,0.02,0.0
7,690449043,men_jeans_ripped,skinny_jeans,39.99,2021-12-13 13:18:24,690449,43,dark_blue_washed_out,skinny_fit,184,31/32,0.98,0.0,0.02,0.0
8,690449043,men_jeans_ripped,skinny_jeans,39.99,2021-12-13 13:18:24,690449,43,denim_blue_trashed,skinny_fit,184,31/32,0.98,0.0,0.02,0.0
9,690449043,men_jeans_ripped,skinny_jeans,39.99,2021-12-13 13:18:24,690449,43,black,skinny_fit,184,31/32,0.98,0.0,0.02,0.0


In [351]:
# save new raw dataset after cleaning it
data.to_csv('products_hm_cleaned.csv')

In [325]:
data.loc[113:115, :]

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,color_name,fit,composition,size_number,size_model
113,974202003,men_jeans_loose,regular_denim_joggers,29.99,2021-12-13 13:18:24,974202,3,light_denim_blue,regular_fit,Cotton 100%,187,NaN
114,974202003,men_jeans_loose,regular_denim_joggers,29.99,2021-12-13 13:18:24,974202,3,denim_blue,regular_fit,Cotton 100%,187,NaN
115,1028865001,men_jeans_relaxed,relaxed_jeans_with_embroidery_detail,49.99,2021-12-13 13:18:24,1028865,1,NaN,NaN,NaN,NaN,NaN


In [311]:
df1[df1[0].isna()]

,0,1,2
115,NaN,NaN,NaN


In [312]:
data.loc[115, :]

product_id                                    1028865001
product_category                       men_jeans_relaxed
product_name        relaxed_jeans_with_embroidery_detail
product_price                                      49.99
scrapy_datetime                      2021-12-13 13:18:24
style_id                                         1028865
color_id                                               1
color_name                                           NaN
fit                                                  NaN
composition                                          NaN
size_number                                          NaN
size_model                                           NaN
Name: 115, dtype: object

In [313]:
data[data['product_id'] == 1028865001]

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,color_name,fit,composition,size_number,size_model
115,1028865001,men_jeans_relaxed,relaxed_jeans_with_embroidery_detail,49.99,2021-12-13 13:18:24,1028865,1,NaN,NaN,NaN,NaN,NaN


In [ ]:
cols = ['product_id','product_category','product_name','product_price','scrapy_datetime','style_id','color_id','color_name','fit','composition','size', 'product_safety']

In [ ]:
#checkin NA
data.isna().sum()

In [ ]:
data[data['product_category'].iloc[471:]]

In [ ]:
# check product_category NA rows
data[data['product_category'].isna()]

In [ ]:
# check unique values in product category
data['product_category'].unique()

In [ ]:
data.info()

In [ ]:
data['composition'].unique()

In [ ]:
data.columns

In [ ]:
# verificando quantidade de cores unicas, provavelmente a granularidade será por cor. 
# Temos 9 cores mas 30 itens, provavelmente duplicados
df_aux.apply(lambda x: len(x.unique()))